In [55]:
import numpy as np
import scipy
import utils
from utils import HouseHolder, QR, SVD

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [56]:
%precision 4

'%.4f'

In [57]:
A = np.array([[0, 0, 0,],
              [0, 0, 0],
              [0, 0, 1],
              [0, 0, 0],
              [2, 5**.5, 0],
              [0, 0, 0],
              [0, 0, 0]], dtype=np.float64)
B, Qt, P = SVD.svd_phaseI(A.T)
print(B)


[[ 0.      2.      0.      0.      0.      0.      0.    ]
 [ 0.     -2.2361  0.      0.      0.      0.      0.    ]
 [ 0.      0.     -1.      0.      0.      0.      0.    ]]


In [58]:
# print(Qt @ A.T @ P)

In [59]:
print(B@B.T)
print(B.T@B)

[[ 4.     -4.4721  0.    ]
 [-4.4721  5.      0.    ]
 [ 0.      0.      1.    ]]
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 9. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


Todo: We may not need to form the reflection matrix H explicitly during Golub-Kahan bidiagonalization.

## Phase-II-A

**Test SVD**

In [60]:
%precision 8

'%.8f'

In [61]:
SVD.svd(A)

(array([[ 0.,  0.,  0.],
        [ 0.,  0., -1.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  0.],
        [ 1.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]),
 array([3.0000000e+00, 1.0000000e+00, 1.6835466e-08]),
 array([[ 0.66666667,  0.74535599,  0.        ],
        [ 0.        ,  0.        , -1.        ],
        [-0.74535599,  0.66666667,  0.        ]]))

In [62]:
scipy.linalg.svd(A, full_matrices=False)

(array([[ 0.,  0.,  0.],
        [ 0.,  0.,  1.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  0.],
        [-1.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]),
 array([3., 1., 0.]),
 array([[-0.66666667, -0.74535599, -0.        ],
        [ 0.        ,  0.        ,  1.        ],
        [-0.74535599,  0.66666667,  0.        ]]))